In [1]:
%matplotlib inline

In [2]:
import numpy as np

## Setup

In [3]:
batch_size = 64

In [4]:
import keras
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Using TensorFlow backend.


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [90]:
'''
X_train=X_train[:1000]
X_test=X_test[:500]
y_train=y_train[:1000]
y_test=y_test[:500]
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
'''

((1000, 28, 28), (1000,), (500, 28, 28), (500,))

In [5]:
# For flow function x: data. Should have rank as 4. 
# In case of grayscale data, the channels axis should have value 1, 
# and in case of RGB data, it should have value 3.

X_train = np.expand_dims(X_train,1)
X_test = np.expand_dims(X_test,1)

In [6]:
X_train.shape

(60000, 1, 28, 28)

In [7]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [8]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
y_train[:5]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [10]:
mean_px = X_train.mean(dtype=np.float32)
std_px = X_train.std(dtype=np.float32)

In [11]:
def norm_input(x):
    return (x-mean_px)/std_px

#### Importing keras modules

In [98]:
from keras.models import Sequential
from keras.layers.core import Lambda, Flatten, Dense 
from keras.optimizers import Adam

#### Pre-processing the images

In [99]:
from keras.preprocessing import image

gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
val_batches = gen.flow(X_test, y_test, batch_size=64)

## NN without hidden layers

In [79]:
nn_without_hidden = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])

nn_without_hidden.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
nn_without_hidden.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                                validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/1
60000/60000 [==============================] - 234s 4ms/step - loss: 0.2509 - acc: 0.9298 - val_loss: 0.3032 - val_acc: 0.9210


## NN with hidden layer

In [81]:
nn_with_hidden = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='tanh'),
        Dense(10, activation='softmax')
        ])

nn_with_hidden.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
nn_with_hidden.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                                validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 1.5433 - acc: 0.8890 - val_loss: 1.0137 - val_acc: 0.9234


## Adding Convolution Layer

In [100]:
from keras.layers import Conv2D,MaxPooling2D

In [101]:
conv_model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Conv2D(32,(3,3), activation='relu'),
        Conv2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        Conv2D(64,(3,3), activation='relu'),
        Conv2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])

conv_model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [102]:
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_9 (Lambda)            (None, 1, 28, 28)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 26, 26)        320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 24, 24)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 10, 10)        18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 8, 8)          36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 4, 4)          0         
__________

In [24]:
conv_model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                                validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.1093 - acc: 0.9667 - val_loss: 0.0589 - val_acc: 0.9797


### Data augmentation

In [25]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)

batches = gen.flow(X_train, y_train, batch_size=64)
val_batches = gen.flow(X_test, y_test, batch_size=64)

In [36]:
conv_model.optimizer.lr=0.001

In [37]:
conv_model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                                validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/8
60000/60000 [==============================] - 21s - loss: 0.0583 - acc: 0.9823 - val_loss: 0.0462 - val_acc: 0.9864
Epoch 2/8
60000/60000 [==============================] - 21s - loss: 0.0467 - acc: 0.9855 - val_loss: 0.0381 - val_acc: 0.9872
Epoch 3/8
60000/60000 [==============================] - 20s - loss: 0.0417 - acc: 0.9870 - val_loss: 0.0330 - val_acc: 0.9896
Epoch 4/8
60000/60000 [==============================] - 21s - loss: 0.0390 - acc: 0.9881 - val_loss: 0.0384 - val_acc: 0.9876
Epoch 5/8
60000/60000 [==============================] - 21s - loss: 0.0361 - acc: 0.9888 - val_loss: 0.0325 - val_acc: 0.9888
Epoch 6/8
60000/60000 [==============================] - 21s - loss: 0.0335 - acc: 0.9895 - val_loss: 0.0275 - val_acc: 0.9914
Epoch 7/8
60000/60000 [==============================] - 21s - loss: 0.0310 - acc: 0.9902 - val_loss: 0.0308 - val_acc: 0.9896
Epoch 8/8
60000/60000 [==============================] - 21s - loss: 0.0302 - acc: 0.9906 - val_loss: 0.0307 - 

In [ ]:
np.argmax(conv_model.predict(np.expand_dims(X_test[2],1)))

In [ ]:
y_test[2]

## Adding Batch Normalization

In [104]:
from keras.layers.normalization import BatchNormalization

In [106]:
conv_batchnorm = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Conv2D(32,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Conv2D(64,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])

conv_batchnorm.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [107]:
conv_batchnorm.optimizer.lr=0.001

In [43]:
conv_batchnorm.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=12, 
                                validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/12
60000/60000 [==============================] - 49s - loss: 0.0346 - acc: 0.9890 - val_loss: 0.0332 - val_acc: 0.9900
Epoch 2/12
60000/60000 [==============================] - 49s - loss: 0.0361 - acc: 0.9892 - val_loss: 0.0435 - val_acc: 0.9875
Epoch 3/12
60000/60000 [==============================] - 48s - loss: 0.0339 - acc: 0.9895 - val_loss: 0.0318 - val_acc: 0.9901
Epoch 4/12
60000/60000 [==============================] - 48s - loss: 0.0309 - acc: 0.9904 - val_loss: 0.0332 - val_acc: 0.9899
Epoch 5/12
60000/60000 [==============================] - 49s - loss: 0.0304 - acc: 0.9905 - val_loss: 0.0267 - val_acc: 0.9907
Epoch 6/12
60000/60000 [==============================] - 48s - loss: 0.0299 - acc: 0.9911 - val_loss: 0.0296 - val_acc: 0.9906
Epoch 7/12
60000/60000 [==============================] - 49s - loss: 0.0268 - acc: 0.9915 - val_loss: 0.0340 - val_acc: 0.9895
Epoch 8/12
60000/60000 [==============================] - 48s - loss: 0.0278 - acc: 0.9912 - val_loss: 0

## Regularization - Dropout

In [109]:
from keras.layers.core import Dropout

In [111]:
conv_batchnorm_dropout = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Conv2D(32,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Conv2D(64,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])

conv_batchnorm_dropout.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [112]:
conv_batchnorm_dropout.optimizer.lr=0.001

In [89]:
conv_batchnorm_dropout.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                                validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 0.0186 - acc: 0.9942 - val_loss: 0.0193 - val_acc: 0.9945
